# Univariate Linear and Polynomial Regression

In this notebook, we will try to solve the simple problem of univariate linear and polynomial regression using two approaches: closed form solution and optimization using gradient descent.

First, we will generate 20 samples of 2-d data from a line and a polynomial in the form $\mathcal{D}=\{x_i, y_i\}_{i=1}^N$, where $N=20$.
We will visualize this data and will try to fit a line and a polynomial using the methods we learnt.

We will fit a linear model using both the closed form solution: $\theta^* = (\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$

Finally, we will fit a polynomial using **gradient descent** as an optimization algorithm. For this, we will use **Pytorch** as the framework.

In all cases, we will visualise the function learnt.

# Data Generation

Let's try to create samples from the line: $y = 3x + 4$. We want to generate some samples but also we want to put in some noise into the mix.

Write some code to generate 30 such samples in the following block. Let's create 2 pytorch 30 dimensional tensors: $X$ which contains 30 values for x and another $y$ which contains the corresponding $y$ values. Of course, make sure to make the data realistic (a.k.a noisy!!).

In [ ]:
import torch

x = torch.tensor(10 * torch.rand(30))

## Write code below to create target outputs for x
## CODE BELOW =======

## ==================

x_linear = x
y_linear = y

In [ ]:
# We want to plot the data and visualise it.

import matplotlib.pyplot as plt
import seaborn as sb

sb.set_style('whitegrid')

plt.plot(x_linear.numpy(), y_linear.numpy(), linestyle="", marker='o')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

Now let's try the same process to create samples from a polynomial: $y = 4x^3 - 2x^2 - 6x + 5$. Instead of getting samples between 0 and 10, let's try getting these samples between 0 and 100.

Write code below to create 50 samples from this polynomial (don't forget to add in some noise to make the samples realistic).

Follow the code above to also plot the samples.

In [ ]:
## Write code below to generate the tensors X and Y following specifications mentioned above.
## CODE BELOW =======

## ==================

x_poly = x
y_poly = y

In [ ]:
## Write code below to plot the data and visualise it.
## CODE BELOW =======


## ==================

## Train model

Next, we want to define the model for fitting the above datasets.

First, let's try to see if we can fit the optimal model using the closed form solution for linear regression on the dataset $x_{linear}, y_{linear}$.

In [ ]:
## CODE BELOW =================
# Don't forget augmenting the matrix of x before applying the closed form solution.
# Consider looking up torch functions for matrix operations.
# Also look up concatenation operations for pytorch tensors.

# Step 1: Augment the matrix x_linear below.
# Sanity check: The shape of the augmented matrix should be [30, 2].


In [ ]:
# Step 2: Compute the term (X^TX)^-1.


In [ ]:
# Step 3: Compute the term X^Ty
# You might need to change the shape of the vector y. Consider looking up torch.unsqueeze


In [ ]:
# Step 4: Compute the optimal parameters using the closed form solution.
# The optimal parameters should have shape [2, 1]


In [ ]:
# Report the values of the optimal parameter vector.
# Are these values close to the parameters of the line we used to create the data?


In [ ]:
# Let's visualise the line we just fit!
# Plot the line along with the data points that you plotted before.


Yaay!! You just successfully fit a line!! Congratulations!

## Gradient Descent

Now that we have explored the closed form solution method of fitting a line on the given data, let's try the more general optimization algorithm: Gradient Descent.

We already have the data. Let's spend some time focusing on the different steps here. It'll be very useful in future as well.

In [ ]:
## Step 1: Define the model

import torch.nn as nn

class LinearModel(nn.Module):
    def __init__(self):
        super(LinearModel, self).__init__()
        self.theta_0 = nn.Parameter(torch.tensor([1.]))
        self.theta_1 = nn.Parameter(torch.tensor([1.]))
    
    def forward(self, x):
        '''
        x can be a minibatch of scalars.
        '''
        return (self.theta_0 + self.theta_1 * x)

linear_model = LinearModel()

In [ ]:
# Step 2: Define the loss function

class SquareLoss(nn.Module):
    def __init__(self):
        super(SquareLoss, self).__init__()
    
    def forward(self, inp, target):
        d = (inp - target) ** 2
        return torch.mean(d)

squared_loss = SquareLoss()

# Note: You can get squared loss (and a lot of other conventional loss functions) as a predefined loss in pytorch.

In [ ]:
# Step 3: Define the optimizer: SGD (Stochastic Gradient Descent)

optimizer = torch.optim.SGD(linear_model.parameters(), lr=0.01)

In [ ]:
# Step 4: Defining the training function

def train(model, x, y, optimizer, loss_fn):
    n_epochs = 1000
    theta_0s = []
    theta_1s = []
    losses = []

    for epoch in range(n_epochs):
        # zero the parameter gradients
        optimizer.zero_grad()
        
        op = model(x)
        loss = loss_fn(op, y)
        
        loss.backward()
        optimizer.step()
        
        theta_0s.append(model.theta_0.item())
        theta_1s.append(model.theta_1.item())
        losses.append(loss.item())
        print (f'==> Epoch {epoch+1}, loss: {loss.item()}, theta_0: {model.theta_0.item()}, theta_1: {model.theta_1.item()}')
        
    return theta_0s, theta_1s, losses

In [ ]:
# Step 5: Training the model

theta_0s, theta_1s, losses = train(linear_model, x_linear, y_linear, optimizer, squared_loss)

# Do the parameters theta_0 and theta_1 look like the ones we used for generating the data at the end of training?
# What happens when we play around with the learning rate for the optimizer? What if we set it to 0.001? What if we set it to 0.1?

In [ ]:
# Step 6: Visualise the line we just learnt.

sb.set_style('whitegrid')

plt.plot(x_linear.numpy(), y_linear.numpy(), linestyle="", marker='o')
plt.plot(x_linear.numpy(), linear_model(x_linear).detach().numpy(), color='r')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

Can you apply and reuse the above code to perform polynomial regression on the generated non-linear toy dataset? 

In [ ]:
## CODE BELOW ============================================
# Note: You might need to try out torch.Adam here instead of torch.SGD. Look them up!
# Note2: Careful when you plot the output of the polynomial, you might need to sort the x's, it's not a line.


In [ ]:
losses = train(polynomial_model, x_poly, y_poly, polynomial_optimizer, squared_loss)

In [ ]:
sb.set_style('whitegrid')

plt.plot(x_poly.numpy(), y_poly.numpy(), linestyle="", marker='o')
x_poly_sort, _ = torch.sort(x_poly)
plt.plot(x_poly_sort.numpy(), polynomial_model(x_poly_sort).detach().numpy(), linewidth=2, color='r')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

Now that you have solved the regression problem on toy settings, try branching out and applying your skills on real-world regression datasets!

Look up a few tutorials on this available online and try to replicate their solutions (or make your own!).
Here's one I found nice: https://towardsdatascience.com/polynomial-regression-using-pytorch-from-scratch-500b7887b0ed. Feel free to follow your own.

Here, you'll find a bunch of regression (and other) datasets: https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table

Homework for next week is to finish up this notebook and also optionally do a polynomial multivariate regression task on a separate dataset above.

Also, next week, we should chat about the **train, validation and test splits**.

Enjoy!